Import necessary libraries


In [213]:
import os
import joblib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

LOAD DATA

In [214]:
#Step 1: Load Data
data = pd.read_excel('Refined/CleanData.xlsx')
print(data.head())

   Year      Total       Male     Female    Ratio  Median Age  Increase Rate  \
0  1960  45954.226  24795.178  21159.049  117.185      18.429          2.485   
1  1961  47060.915  25363.721  21697.194  116.899      18.360          2.277   
2  1962  48161.841  25930.189  22231.652  116.636      18.271          2.347   
3  1963  49325.050  26526.519  22798.530  116.352      18.183          2.425   
4  1964  50552.592  27153.709  23398.883  116.047      18.084          2.490   

   Life Expectancy  Inflation  Unemployment           GDP  Homicides  
0           43.355   6.947368           0.4  3.749265e+09   3.651252  
1           44.180   1.640420           0.4  4.118648e+09   3.651252  
2           45.009  -0.516462           0.4  4.310164e+09   3.651252  
3           46.318   1.456488           0.4  4.630827e+09   3.651252  
4           47.360   4.179587           0.4  5.204956e+09   3.651252  


population 

In [215]:
X = data['Year'].values.reshape(-1, 1)
y = data['Total'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

population_model = LinearRegression()
population_model.fit(X_train, y_train)

y_pred = population_model.predict(X_test)

print("Accuracy: ", population_model.score(X_test, y_test).round(5) * 100, "%")

Accuracy:  98.433 %


gdp

In [216]:
X = data['Year'].values.reshape(-1, 1)
y = np.log(data['GDP'].values)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

poly_features = PolynomialFeatures(degree=2)
X_train_poly = poly_features.fit_transform(X_train)
X_test_poly = poly_features.transform(X_test)

gdp_model = LinearRegression()
gdp_model.fit(X_train_poly, y_train)

print("Accuracy:", gdp_model.score(X_test_poly, y_test).round(5) * 100, "%")

Accuracy: 99.612 %


unemployment

In [217]:
X = data['Year'].values.reshape(-1, 1)
y = np.log(data['Unemployment'].values)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

poly_features = PolynomialFeatures(degree=2)
X_train_poly = poly_features.fit_transform(X_train)
X_test_poly = poly_features.transform(X_test)

unemployment_model = LinearRegression()
unemployment_model.fit(X_train_poly, y_train)

print("Accuracy", unemployment_model.score(X_test_poly, y_test).round(5) * 100, "%")

Accuracy 69.321 %


inflation

In [227]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# Load data
data = pd.read_excel('Refined/CleanData.xlsx')

# Filter data for relevant years
data = data[data['Year'] >= 1990]

# Prepare data
X = data[['Year']].values
y = data['Inflation'].values

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Decision Tree Regressor model
dt_model = DecisionTreeRegressor(random_state=42)
dt_model.fit(X_train, y_train)

# Evaluate model
y_pred_dt = dt_model.predict(X_test)
accuracy_dt = r2_score(y_test, y_pred_dt)
print(f"Decision Tree Regressor Accuracy: {accuracy_dt * 100:.2f}%")

# Predict inflation for the year 2025
Year_2025 = [[2025]]
predicted_inflation_2025 = dt_model.predict(Year_2025)
print(f"Predicted Inflation in 2025: {predicted_inflation_2025[0]:.2f}")


Decision Tree Regressor Accuracy: -103.95%
Predicted Inflation in 2025: 9.50


homicides

In [219]:
# Load and filter data
data = pd.read_excel('Refined/CleanData.xlsx')
data = data[data['Year'] >= 1990]

# Feature and target variables
X = data['Year'].values.reshape(-1, 1)
y = np.log(data['Homicides'].values)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Polynomial features
poly_features = PolynomialFeatures(degree=2)
X_train_poly = poly_features.fit_transform(X_train)
X_test_poly = poly_features.transform(X_test)

# Model training
homicides_model = RandomForestRegressor(n_estimators=100, random_state=42)
homicides_model.fit(X_train_poly, y_train)

# Model evaluation
print("Accuracy:", homicides_model.score(X_test_poly, y_test).round(5) * 100, "%")


Accuracy: 95.883 %


final prediction

In [220]:
# Ask for the input year
Year = int(input("Enter the year for prediction: "))
features_for_prediction = [[Year]]

# Predict population
predictions = population_model.predict(features_for_prediction)
print(f"Predicted Population in {Year}: {int(predictions[0])}")

# Predict unemployment
predictions = np.exp(unemployment_model.predict(poly_features.transform(features_for_prediction)))
print(f"Predicted Unemployment in {Year}: {int(predictions[0])}")

# Predict GDP
predictions = np.exp(gdp_model.predict(poly_features.transform(features_for_prediction)))
print(f"Predicted GDP in {Year}: {int(predictions[0])}")

# Predict homicides
predictions = np.exp(homicides_model.predict(poly_features.transform(features_for_prediction)))
print(f"Predicted Homicides in {Year}: {predictions[0]:.2f}")

# Predict inflation
predicted_inflation = inflation_model.predict(poly_features.transform(features_for_prediction))
print(f"Predicted Inflation in {Year}: {predicted_inflation[0]:.2f}")


Predicted Population in 2025: 237210
Predicted Unemployment in 2025: 7
Predicted GDP in 2025: 526597842345
Predicted Homicides in 2025: 3.93
Predicted Inflation in 2025: 6.18
